In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


GETTING A PART OF THE PUBLAYNET DATASET

In [2]:
# importing prerequisites
import sys
import requests
import tarfile
import json
import numpy as np
from os import path
from PIL import Image
from PIL import ImageFont, ImageDraw
from glob import glob
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
fname = 'examples.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-publaynet/1.0.0/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)

6461141

In [4]:
# Extracting the dataset
tar = tarfile.open(fname)
tar.extractall()
tar.close()

In [5]:
# Verifying the file was extracted properly
data_path = "examples/"
path.exists(data_path)

True

# Install detectron2

In [6]:
!pip install pyyaml==5.1
# This is the current pytorch version on Colab. Uncomment this if Colab changes its pytorch version
!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

     |████████████████████████████████| 274 kB 4.3 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=ade6c9e18032ebcad22f75985228b10f75b2b7481e9747dd67beabc1b4250050
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
     |████████████████████████████████| 6.4 MB 655 kB/s 
     |████████████████████████████████| 130 kB 4.2 MB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
     |████████████████████████████████| 74 kB 3.1 MB/s 
     |████████████████████████████████| 145 kB 42.7 MB/s 
     |████████████████████████████████| 743 kB 36.7 MB/s

In [1]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [2]:
#!wget https://github.com/facebookresearch/detectron2.git
!git clone https://github.com/facebookresearch/Detectron2.git
!pip install -e detectron2

Cloning into 'Detectron2'...
remote: Enumerating objects: 13028, done.
remote: Counting objects: 100% (529/529), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 13028 (delta 307), reused 436 (delta 270), pack-reused 12499
Receiving objects: 100% (13028/13028), 5.59 MiB | 10.75 MiB/s, done.
Resolving deltas: 100% (9255/9255), done.
ERROR: detectron2 is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+http, svn+https, svn+svn, svn+file).


In [3]:
#hpanwar's repo
!git clone https://github.com/hpanwar08/detectron2.git

Cloning into 'detectron2'...
remote: Enumerating objects: 2152, done.
remote: Total 2152 (delta 0), reused 0 (delta 0), pack-reused 2152
Receiving objects: 100% (2152/2152), 4.64 MiB | 12.83 MiB/s, done.
Resolving deltas: 100% (1355/1355), done.


In [4]:
rm -rf build

In [5]:
rm -rf build/ **/*.so

In [6]:
%cd /content/Detectron2
!python setup.py build develop

/content/Detectron2
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/detectron2
copying detectron2/__init__.py -> build/lib.linux-x86_64-3.7/detectron2
creating build/lib.linux-x86_64-3.7/tools
copying tools/benchmark.py -> build/lib.linux-x86_64-3.7/tools
copying tools/visualize_data.py -> build/lib.linux-x86_64-3.7/tools
copying tools/visualize_json_results.py -> build/lib.linux-x86_64-3.7/tools
copying tools/analyze_model.py -> build/lib.linux-x86_64-3.7/tools
copying tools/train_net.py -> build/lib.linux-x86_64-3.7/tools
copying tools/convert-torchvision-to-d2.py -> build/lib.linux-x86_64-3.7/tools
copying tools/__init__.py -> build/lib.linux-x86_64-3.7/tools
copying tools/lazyconfig_train_net.py -> build/lib.linux-x86_64-3.7/tools
copying tools/plain_train_net.py -> build/lib.linux-x86_64-3.7/tools
copying tools/lightning_train_net.py -> build/lib.linux-x86_64-3.7/tools
creating build/lib.linux-x86_64-3.7/detectr

THE FOLLOWING SCRIPT NEEDS TO BE RUN AS A .PY FILE USING THE COMMAND GIVEN IN THE LAST CELL

In [ ]:
import logging
import os
import datetime
from collections import OrderedDict
import torch

import detectron2
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    verify_results,
)
from detectron2.modeling import GeneralizedRCNNWithTTA


from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger

from detectron2.evaluation import COCOEvaluator, inference_on_dataset, print_csv_format
from detectron2.data import build_detection_test_loader

logger = logging.getLogger("detectron2")


class Trainer(DefaultTrainer):

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        evaluator = COCOEvaluator(dataset_name)
        val_loader = build_detection_test_loader(cfg, dataset_name)
        #print(inference_on_dataset(trainer.model,val_loader, evaluator))
        results=inference_on_dataset(trainer.model,val_loader,evaluator)
        print_csv_format(results)
        return results



def main(args):
    print("CALLED")

    if args.eval_only:
      Trainer.build_evaluator(cfg,"my_dataset_val")
    


if __name__ == "__main__":
    args = default_argument_parser().parse_args()
    output_dir = "/content/drive/MyDrive/TESTING"
    os.makedirs(output_dir, exist_ok=True)


    register_coco_instances("my_dataset_train", {}, "/content/drive/MyDrive/Layout-COCO/train_coco/train.json", "/content/drive/MyDrive/Layout-COCO/train_coco/training")
    register_coco_instances("my_dataset_val", {}, "/content/drive/MyDrive/Layout-COCO/validation_coco/val.json", "/content/drive/MyDrive/Layout-COCO/validation_coco/data")
    register_coco_instances("my_dataset_test", {}, "/content/drive/MyDrive/Layout-COCO/test_coco/test.json", "/content/drive/MyDrive/Layout-COCO/test_coco/data")
    register_coco_instances("my_dataset_val1", {}, "/content/examples/samples.json", "/content/examples/data1") 


    cfg = get_cfg()
    cfg.OUTPUT_DIR = output_dir

    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)


    #cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
    #cfg.merge_from_file("/content/detectron2/configs/DLA_mask_rcnn_X_101_32x8d_FPN_3x.yaml")

    cfg.DATASETS.TRAIN = ("my_dataset_train",)
    cfg.DATASETS.TEST = ("my_dataset_test",)
    cfg.DATALOADER.NUM_WORKERS = 2

    #cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("DLA_mask_rcnn_X_101_32x8d_FPN_3x.yaml")
    #cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/model_0005499_new.pth"

    cfg.SOLVER.IMS_PER_BATCH = 10
    cfg.SOLVER.BASE_LR = 0.003  # pick a good LR
    cfg.SOLVER.MAX_ITER = 10    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
    cfg.SOLVER.STEPS = []        # do not decay learning rate
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 10   # faster, and good enough for this toy dataset (default: 512)
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
    # NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

    cfg.TEST.EVAL_PERIOD = 1000
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = DefaultTrainer(cfg) 
    #trainer = CocoTrainer(cfg)
    trainer.resume_or_load(resume=False)
    #trainer.train()
    launch(
        main,
        args.num_gpus,
        num_machines=args.num_machines,
        machine_rank=args.machine_rank,
        dist_url=args.dist_url,
        args=(args,),
    )

In [7]:
!python /content/detectron2/tools/train_net_dla.py \
	--config-file /content/detectron2/configs/DLA_mask_rcnn_X_101_32x8d_FPN_3x.yaml \
	--eval-only MODEL.WEIGHTS /content/drive/MyDrive/model_0005499_new.pth

Config '/content/detectron2/configs/DLA_mask_rcnn_X_101_32x8d_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
[09/19 15:27:52 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7)